In [1]:
#Importing all the packages
import pandas as pd
import matplotlib.pyplot as plt
plt.style.use('seaborn')
%matplotlib inline
from cobra.io import read_sbml_model
from cobra.io import write_sbml_model

In [2]:
#Importing the model "iCW773"with changed names --> iCW773_test7.xml
model = read_sbml_model('iCW773_test7.xml') #for test file e.g. trp- was replaced by trp_ to enable calling of the metabolite

No objective coefficients in model. Unclear what should be optimized


In [3]:
model.compartments #no compartment are utilized by the base model.

{'': ''}

In [4]:
for metabolite in model.metabolites:
    if '_e' in metabolite.id:
        metabolite.compartment='e' #finds the extracellular metabolites, assigns to compartment e

In [5]:
model.compartments

{'': '', 'e': ''}

In [6]:
for metabolite in model.metabolites:
    if '_c' in metabolite.id:
        metabolite.compartment='c' #finds the cytoplasmic metabolites, assigns to compartment c

In [7]:
model.compartments #shows that all metabolites are assigned to either c or e

{'c': '', 'e': ''}

In [8]:
#for metabolite in model.metabolites:
 #   if not metabolite.compartment:
  #      metabolite.compartment='c' #assigns the compartment c to anything that does not have a compartment yet, to remove error in escher and match the code below.

Adding metabolites to the model

In [9]:
from cobra import Reaction, Metabolite
tryptamine = Metabolite(id = 'tryp_c', compartment='c', name='tryptamine', formula='C10H12N2')
hydroxytryptamine = Metabolite(id='_4htryp_c', compartment='c', name='4-Hydroxytryptamine', formula='C10H12N2O')
norbaeocystin = Metabolite(id='nbc_c' , compartment='c', name='Norbaeocystin', formula='C10H13N2O4P')
psilocybin = Metabolite(id='psi_c', compartment='c', name='Psilocybin', formula='C12H17N2O4P')
psilocin = Metabolite(id='psii_c', compartment='c', name='psilocin', formula='C12H16N2O')
baeocystin = Metabolite(id='baecyst_c', compartment='c', name='Baeocystin', formula='C11H15N2O4P')
psilocybin_e = Metabolite(id='psi_e', compartment='e', name='Psilocybin_e', formula='C12H17N2O4P')

## Defining the reactions

In [10]:
L_tryptophan_decarboxylase = Reaction('CrTdc')
tryptamine_4_monooxygenase = Reaction('psiH')
_4_hydroxytryptamine_kinase = Reaction('psiK')
psilocybin_synthase = Reaction('psiM')
psilocybin_synthase2 = Reaction('psiM2')
pho_or_spon = Reaction('pho') # reaction is either spontaneous or carried out by phosphatases
_4_hydroxytryptamine_kinase2 = Reaction('psiK2')
psitransportpathway = Reaction('psitpp') #remove the psilocybin from the cell, transporter such as PSIT1_PSICU

## Adding the metabolites to the reactions


In [11]:
L_tryptophan_decarboxylase.add_metabolites(({model.metabolites.trp_L_c: -1,
                              tryptamine: 1, 
                              model.metabolites.co2_c: 1, 
                            }))

In [12]:
tryptamine_4_monooxygenase.add_metabolites({tryptamine: -1,
                              model.metabolites.nadph_c: -2,           
                              model.metabolites.o2_c: -1,
                              hydroxytryptamine: 1,
                              model.metabolites.nadp_c:2,
                              model.metabolites.h2o_c: 1,              
                             })

In [13]:
_4_hydroxytryptamine_kinase.add_metabolites({hydroxytryptamine: -1,
                              model.metabolites.atp_c: -1,
                              norbaeocystin: 1, 
                              model.metabolites.adp_c: 1,
                             })

In [14]:
psilocybin_synthase.add_metabolites(({norbaeocystin: -1,
                             model.metabolites.amet_c: -1, 
                             baeocystin: 1, 
                             model.metabolites.ahcys_c: 1,
                             }))

In [15]:
psilocybin_synthase2.add_metabolites(({baeocystin: -1,
                             model.metabolites.amet_c: -1, 
                             psilocybin: 1, 
                             model.metabolites.ahcys_c: 1,
                             }))

In [16]:
pho_or_spon.add_metabolites(({psilocybin: -1,
                             model.metabolites.h_c: -1,
                             psilocin: 1,
                             model.metabolites.pi_c: 1,
                             }))

In [17]:
_4_hydroxytryptamine_kinase2.add_metabolites(({psilocin: -1,
                             model.metabolites.atp_c: -1, 
                             psilocybin: 1, 
                             model.metabolites.adp_c: 1, 
                             }))

In [18]:
psitransportpathway.add_metabolites(({psilocybin: -1,
                             psilocybin_e: 1, 
                             })) #can be passive transport, https://www.uniprot.org/uniprot/P0DPB1

## Adding the reactions to the model

In [19]:
model.add_reactions([L_tryptophan_decarboxylase]) #
model.add_reactions([tryptamine_4_monooxygenase]) #
model.add_reactions([_4_hydroxytryptamine_kinase]) #
model.add_reactions([_4_hydroxytryptamine_kinase2]) #
model.add_reactions([psilocybin_synthase]) #
model.add_reactions([psilocybin_synthase2]) #
model.add_reactions([pho_or_spon]) #
model.add_reactions([psitransportpathway]) #

In [20]:
#model.add_boundary(model.metabolites.tryp_c, type='demand')
#model.add_boundary(model.metabolites._4htryp_c, type='demand')
#model.add_boundary(model.metabolites.nbc_c, type='demand')
#model.add_boundary(model.metabolites.psi_c, type='demand') #removed demand function
model.add_boundary(model.metabolites.psi_e, type='exchange') #use exchange function for product instead
#model.add_boundary(model.metabolites.amet_c, type='demand')
model.add_boundary(model.metabolites.ahcys_c, type='demand')
#model.add_boundary(model.metabolites.psii_c, type='demand');

Reaction identifier,DM_ahcys_c
Name,S-AdenosyL_L_homocysteine demand
Memory address,0x0206bb5f44a8
Stoichiometry,ahcys_c --> S-AdenosyL_L_homocysteine -->
GPR,
Lower bound,0
Upper bound,1000.0


For the production of psilocybin, SAH (amet_c) is produced from SAM (ahcys_c), thus a demand function is necessary. 

In [21]:
model.reactions.EX_psi_e.bounds = 0,1000

In [22]:
write_sbml_model(model, "iCW773_psilocybin.xml")